In [47]:
import psycopg2 as ps
import ipyleaflet as ipyl
import ipywidgets as ipyw
import pandas as pd
import json


In [48]:
'''
Preliminary works:
To extract OpenStreetMap data that we want to display here we created filter following transnet project as below:
1. All power nodes/ways of relations tagged with power=*"
2. All relations tagged with route=power"
3. All ways and its corresponding nodes tagged with power=*"
Than we merge all extracts data before converts it using osm2pgsql to postGIS-enabled PostgreSQL databases.

This section handle database connection to our postgreSQL database.
Since ipyleaflet working with GeoJSON format, we use query database that convert our extracted database to JSON format.
'''

conn_string = "dbname='gis' user='postgres' password='123456' host='localhost'"
conn = ps.connect(conn_string)
cur = conn.cursor()
query = """SELECT 
               * 
           FROM 
               planet_osm_line 

"""
columns = """
        operator,
        name,
        highway,
        amenity,
        public_transport,
        railway,
        power,
        bicycle,
        barrier
"""
query_json = """
SELECT row_to_json(fc)
 FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
 FROM (SELECT 'Feature' As type
    , ST_AsGeoJSON(ST_Transform(lg.way, 4326))::json As geometry
    , row_to_json((SELECT l FROM (SELECT {}) As l
      )) As properties
   FROM planet_osm_line As lg   ) As f )  As fc limit 10;
""".format(columns)

cur.execute(query_json)
d = cur.fetchone()

df = pd.read_sql(query, conn)
conn.close()

In [49]:
#Loading map area in Berlin
map = ipyl.Map(center=[52.52, 13.45], zoom=9)
label = ipyw.Label(layout=ipyw.Layout(width='100%'))

#Loading GeoJSON to display extracted data on the map
layer = ipyl.GeoJSON(data=d[0], hover_style={'fillColor': 'red'})

#Function handling hover action to show label properties below the map
def hover_handler(event=None, id=None, properties=None):
    s = ""
    for key, value in properties.items():
         s = s + str(key) +":" + str(value) + ";\t"
    label.value = s
    
layer.on_hover(hover_handler)
map.add_layer(layer)


ipyw.VBox([map, label])
